In [1]:
import pandas as pd

from utils import tag_mail

df = pd.read_json(r'data/gmail_turc_basic.txt', encoding = 'utf-8')

df['cat'] = df.headers.apply(tag_mail)

df.loc[df.cat=='uber']

,snippet,date,headers,body,cat
67,15€ sur votre première commande Uber Eats Essa...,2020-03-05 09:47:02,"[{'name': 'From', 'value': 'Uber <noreply@uber...",15€ sur votre première commande Uber Eats ESSA...,uber
131,Envie de bons petits plats. Par ici les économ...,2020-02-27 09:47:02,"[{'name': 'From', 'value': 'Uber <noreply@uber...",Envie de bons petits plats. PAR ICI LES ÉCONOM...,uber
188,"Profitez-en ! Bonjour Etienne, Vous n&#39;avez...",2020-02-17 15:50:11,"[{'name': 'From', 'value': 'Uber <uber@uber.co...","Profitez-en ! Bonjour Etienne,\n\nVous n’avez ...",uber
304,50% sur votre 1ère commande Uber Eats. Activez...,2020-02-02 15:00:23,"[{'name': 'From', 'value': 'Uber <uber@uber.co...",50% sur votre 1ère commande Uber Eats. ACTIVEZ...,uber
358,"Des délices à portée de main, en quelques clic...",2020-01-25 15:00:22,"[{'name': 'From', 'value': 'Uber <uber@uber.co...","Des délices à portée de main, en quelques clic...",uber
378,"Vos plats préférés, livrés où vous voulez Nous...",2020-01-22 15:00:21,"[{'name': 'From', 'value': 'Uber <uber@uber.co...","Vos plats préférés, livrés où vous voulez NOUS...",uber
506,5€ de réduction pour chaque parrainage Économi...,2020-01-07 09:58:47,"[{'name': 'From', 'value': 'Uber <uber@uber.co...",5€ de réduction pour chaque parrainage ÉCONOMI...,uber
552,Utilisez votre code EATSENFETE ! Régalez-vous ...,2020-01-02 10:53:06,"[{'name': 'From', 'value': 'Uber <uber.france@...",Utilisez votre code EATSENFETE ! RÉGALEZ-VOUS ...,uber
677,Utilisez votre code EATSENFETE ! Un cadeau sou...,2019-12-19 13:29:57,"[{'name': 'From', 'value': 'Uber <uber.france@...",Utilisez votre code EATSENFETE ! UN CADEAU SOU...,uber
732,Votre expérience aux aéroports avec Uber Bonjo...,2019-12-11 13:51:12,"[{'name': 'From', 'value': 'Uber <uber@uber.co...",VOTRE EXPÉRIENCE AUX AÉROPORTS AVEC UBER\nBon...,uber


In [7]:
import re
from datetime import datetime

df_uber = df.loc[df.cat=='uber']

res = []

for date, el in df_uber.loc[:,['date', 'body']].itertuples(index=False):
    tmp = {}
    # place
    try:
        departure, destination = re.findall(r'\d\d:\d\d (.*?) \d\d:\d\d (.*?) Invitez', el)[0]
        tmp['departure'] = departure
        tmp['destination'] = destination
    except:
        continue
    
    # price    
    try:
        price = re.findall(r' Total: \d{1,1000},\d{2,1000} € ',el )[0]
        price = re.findall(r'\d{1,1000},\d{2,1000}', price)[0]
        tmp['price'] = price
    except:
        price = None
        tmp['price'] = price
        pass
    
    # distance
    try:
        distance = re.findall(r'\d{1,1000}.{0,1}\d{0,1000} km', el)[0]
        tmp['distance'] = distance
    except:
        distance = None
        tmp['distance'] = distance
        pass
    
    # date
    try:
        horaire = re.findall(r'\d\d:\d\d', el)
        start = datetime.strptime(horaire[0], '%H:%M')
        end = datetime.strptime(horaire[1], '%H:%M')
        start = start.replace(year = date.year, month = date.month, day = date.day)
        end = end.replace(year = date.year, month = date.month, day = date.day)
        tmp['start'] = start
        tmp['end'] = end
    except:
        start = None
        end = None
        tmp['start'] = start
        tmp['end'] = end
        pass
    
    res += [tmp]
    print(tmp)
    print('\n\n')
    print('#######################################################')
    print('\n\n')
    
return res

{'departure': '93290 Tremblay-en-France, France', 'destination': '21 Rue Maximilien Robespierre, 91120 Palaiseau, France', 'price': '74,92', 'distance': '58.08 km', 'start': datetime.datetime(2019, 12, 10, 14, 32), 'end': datetime.datetime(2019, 12, 10, 15, 43)}



#######################################################



{'departure': '21 Rue Maximilien Robespierre, 91120 Palaiseau, France', 'destination': 'Gare SNCF TGV, 77990 Le Mesnil-Amelot, France', 'price': '77,35', 'distance': '55.13 km', 'start': datetime.datetime(2019, 12, 8, 10, 8), 'end': datetime.datetime(2019, 12, 8, 10, 58)}



#######################################################





[{'departure': '93290 Tremblay-en-France, France',
  'destination': '21 Rue Maximilien Robespierre, 91120 Palaiseau, France',
  'price': '74,92',
  'distance': '58.08 km',
  'start': datetime.datetime(2019, 12, 10, 14, 32),
  'end': datetime.datetime(2019, 12, 10, 15, 43)},
 {'departure': '21 Rue Maximilien Robespierre, 91120 Palaiseau, France',
  'destination': 'Gare SNCF TGV, 77990 Le Mesnil-Amelot, France',
  'price': '77,35',
  'distance': '55.13 km',
  'start': datetime.datetime(2019, 12, 8, 10, 8),
  'end': datetime.datetime(2019, 12, 8, 10, 58)}]

In [3]:
df_uber.loc[738].body

" Total: 74,92 € mar., déc. 10, 2019 Etienne, merci d'avoir utilisé Uber Nous espérons que vous avez apprécié votre course cet après-midi. Total 74,92 € Prix du trajet 74,92 € Sous-total 74,92 € Montant facturé Bon : Uber rides for Etienne Turc 74,92 € Pour consulter vos reçus (et vos factures le cas échéant), ou pour obtenir plus d'informations, rendez-vous sur votre espace personnel. xidb5307955-890f-4898-9f52-0c5555da82d7 pGvlI2ANUbXFfyEOgxta1RMV082993 Votre chauffeur était Hafedh 4.91 Note Hafedh est apprécié pour : \nExcellent Service Comment s'est passée votre course ? Note ou pourboire Note ou pourboire Reçu émis au nom de DH TRANSPORT DE PERSONNES UberX 58.08 km | 1 h 11 min 14:32 93290 Tremblay-en-France, France 15:43 21 Rue Maximilien Robespierre, 91120 Palaiseau, France Invitez vos amis et votre famille. Bénéficiez d’une réduction de 5 € sur votre prochaine course en invitant un ami à découvrir Uber.Code de parrainage : 01i3pmSignaler la perte d'un objet ❯contacter l'assista

In [41]:
t = ' Total: 74,92 € '

In [43]:
re.findall('\d{1,1000},\d{2,1000}', t)[0]

'74,92'

In [85]:
for date, mail in df_uber.loc[:,['date', 'body']].itertuples(index=False):
    print(date)

2019-12-10 14:43:37
2019-12-08 09:58:55
2018-11-23 18:41:55


In [82]:
for a, b in df_uber.itertuples(index=False):
    print(a)

ValueError: too many values to unpack (expected 2)